## Introducción – Caso ejercicio

**Empresa de micromovilidad:**

Luego de un exitoso lanzamiento en el mercado, ahora es el momento de expandir
nuestras operaciones en Latam. Estamos enfrentando un gran desafío y queremos
contar con el mejor talento para lograrlo.

**Ejercicio 1: Análisis de Datos**

En este ejercicio, recibirás un archivo .csv que contiene información sobre 30,000
registros de transacciones. Aunque entender el significado de cada campo es parte del
reto, te damos dos recomendaciones clave:
1. Los campos "id" contienen identificadores únicos, en formato alfanumérico en
lugar de nombres.
2. Las horas están representadas en UTC, por lo que debes tener cuidado con los
valores de precios, distancias y duración de las transacciones.

**¿Qué esperamos?**

Queremos un análisis exhaustivo (no trivial) que nos ayude a tomar mejores decisiones.
Te pedimos que nos presentes tu interpretación de los datos y realices propuestas
basadas en tu análisis. Por ejemplo:
* ¿Dónde y en qué horarios deberíamos desplegar nuestros recursos?
* ¿Cómo es nuestra tasa de ocupación? ¿Cómo podríamos mejorarla?
* ¿Cómo optimizar la eficiencia operativa de nuestros procesos?
* ¿Nuestros clientes son lo suficientemente leales al servicio? Analiza hipótesis y conclusiones.
* ¿Qué clientes deberíamos retener? ¿Qué clientes deberíamos reactivar?
* Después de analizar estos datos, ¿qué plan de crecimiento o decisiones tomarías
para fortalecer la operación?

## Solución

Verificamos que la versión de Python sea superior a Python 3.7

In [1]:
import sys

assert sys.version_info >= (3, 7)

Vamos a considerar Scikit-Learn superior a 1.0.1

In [2]:
from packaging import version
import sklearn 

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")